In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
xmin=0
xmax=1


x = np.linspace(0,1,32)
grid = np.meshgrid(x,x)

X, Y = np.mgrid[xmin:xmax:32j, xmin:xmax:32j]
positions = np.vstack([X.ravel(), Y.ravel()])

positions = positions.T
dim = positions.shape[0]
ones = np.linspace(0,dim-1,dim).astype(int)

res = pd.DataFrame(positions)
#res = np.column_stack([ones.T,positions])

print(res)

        0         1
0     0.0  0.000000
1     0.0  0.032258
2     0.0  0.064516
3     0.0  0.096774
4     0.0  0.129032
...   ...       ...
1019  1.0  0.870968
1020  1.0  0.903226
1021  1.0  0.935484
1022  1.0  0.967742
1023  1.0  1.000000

[1024 rows x 2 columns]


In [3]:
#print(res[res.index==1])
#print(1 in res.iloc[31:32,:].values)
print(res.iloc[3].values)

[0.         0.09677419]


In [4]:
res[res.index==31]

,0,1
31,0.0,1.0


In [3]:
#generate the general patch
def patches(positions):
    result = []
    #result = pd.DataFrame(data=a)
    #result=result.T
    for ind in range(0,dim-1):
        if not( 1 in res.iloc[ind:ind+1,:].values):
            upper = [ind,ind+1,ind+33]
            lower = [ind,ind+32, ind+33]
            #upper.append([])
            result.append(upper)
            result.append(lower)
    return pd.DataFrame(result)

patch = patches(res)
print(patch)

        0     1     2
0       0     1    33
1       0    32    33
2       1     2    34
3       1    33    34
4       2     3    35
...   ...   ...   ...
1917  988  1020  1021
1918  989   990  1022
1919  989  1021  1022
1920  990   991  1023
1921  990  1022  1023

[1922 rows x 3 columns]


In [4]:
## generate the nearest neighbours

def neighbour_ind(patches, points):
    list = []
    for ind in points.index:
        obj = []
        for pat in patches.index:
            if (ind in patches.iloc[pat].values):
                obj.append(pat)
                #obj.append(patches.iloc[pat].values)
                print(pat, ind)
        #obj = [val for sublist in obj for val in sublist]
        #obj = [int(a) for a in obj]
        list.append(obj)
    return pd.DataFrame(list)
pat_neighbour = neighbour_ind(patch,res)
print(pat_neighbour)

0 0
1 0
0 1
2 1
3 1
2 2
4 2
5 2
4 3
6 3
7 3
6 4
8 4
9 4
8 5
10 5
11 5
10 6
12 6
13 6
12 7
14 7
15 7
14 8
16 8
17 8
16 9
18 9
19 9
18 10
20 10
21 10
20 11
22 11
23 11
22 12
24 12
25 12
24 13
26 13
27 13
26 14
28 14
29 14
28 15
30 15
31 15
30 16
32 16
33 16
32 17
34 17
35 17
34 18
36 18
37 18
36 19
38 19
39 19
38 20
40 20
41 20
40 21
42 21
43 21
42 22
44 22
45 22
44 23
46 23
47 23
46 24
48 24
49 24
48 25
50 25
51 25
50 26
52 26
53 26
52 27
54 27
55 27
54 28
56 28
57 28
56 29
58 29
59 29
58 30
60 30
61 30
60 31
1 32
62 32
63 32
0 33
1 33
3 33
62 33
64 33
65 33
2 34
3 34
5 34
64 34
66 34
67 34
4 35
5 35
7 35
66 35
68 35
69 35
6 36
7 36
9 36
68 36
70 36
71 36
8 37
9 37
11 37
70 37
72 37
73 37
10 38
11 38
13 38
72 38
74 38
75 38
12 39
13 39
15 39
74 39
76 39
77 39
14 40
15 40
17 40
76 40
78 40
79 40
16 41
17 41
19 41
78 41
80 41
81 41
18 42
19 42
21 42
80 42
82 42
83 42
20 43
21 43
23 43
82 43
84 43
85 43
22 44
23 44
25 44
84 44
86 44
87 44
24 45
25 45
27 45
86 45
88 45
89 45
26 46
27 46
29 

In [5]:
pat_neighbour = pat_neighbour.fillna(-1)
pat_neighbour = pat_neighbour.astype(int)

print(pat_neighbour.head())

   0  1  2  3  4  5
0  0  1 -1 -1 -1 -1
1  0  2  3 -1 -1 -1
2  2  4  5 -1 -1 -1
3  4  6  7 -1 -1 -1
4  6  8  9 -1 -1 -1


In [25]:
def neighbour_p(patches, points):
    list = []
    for ind in points.index:
        obj = []
        for pat in patches.index:
            if (ind in patches.iloc[pat].values):
                #obj.append(pat)
                loc_obj = [a for a in patches.iloc[pat].values
                       if a !=ind
                       ]
                obj.append(loc_obj)
                #obj.remove(ind)
                print(pat, ind)
        #obj = [val for sublist in obj for val in sublist]
        #obj = [int(a) for a in obj]
        #obj.remove(ind)
        list.append(obj)
    return pd.DataFrame(list)
point_neighbour = neighbour_p(patch,res)

print(point_neighbour.head())

0 0
1 0
0 1
2 1
3 1
2 2
4 2
5 2
4 3
6 3
7 3
6 4
8 4
9 4
8 5
10 5
11 5
10 6
12 6
13 6
12 7
14 7
15 7
14 8
16 8
17 8
16 9
18 9
19 9
18 10
20 10
21 10
20 11
22 11
23 11
22 12
24 12
25 12
24 13
26 13
27 13
26 14
28 14
29 14
28 15
30 15
31 15
30 16
32 16
33 16
32 17
34 17
35 17
34 18
36 18
37 18
36 19
38 19
39 19
38 20
40 20
41 20
40 21
42 21
43 21
42 22
44 22
45 22
44 23
46 23
47 23
46 24
48 24
49 24
48 25
50 25
51 25
50 26
52 26
53 26
52 27
54 27
55 27
54 28
56 28
57 28
56 29
58 29
59 29
58 30
60 30
61 30
60 31
1 32
62 32
63 32
0 33
1 33
3 33
62 33
64 33
65 33
2 34
3 34
5 34
64 34
66 34
67 34
4 35
5 35
7 35
66 35
68 35
69 35
6 36
7 36
9 36
68 36
70 36
71 36
8 37
9 37
11 37
70 37
72 37
73 37
10 38
11 38
13 38
72 38
74 38
75 38
12 39
13 39
15 39
74 39
76 39
77 39
14 40
15 40
17 40
76 40
78 40
79 40
16 41
17 41
19 41
78 41
80 41
81 41
18 42
19 42
21 42
80 42
82 42
83 42
20 43
21 43
23 43
82 43
84 43
85 43
22 44
23 44
25 44
84 44
86 44
87 44
24 45
25 45
27 45
86 45
88 45
89 45
26 46
27 46
29 

In [6]:

def neighbour_point(patches, points):
    list = []
    for ind in points.index:
        obj = []
        for pat in patches.index:
            if (ind in patches.iloc[pat].values):
                #obj.append(pat)
                loc_obj = [res.iloc[a].values for a in patches.iloc[pat].values
                       if a !=ind
                       ]
                obj.append(loc_obj)
                #obj.remove(ind)
                print(res.iloc[ind].values)
        #obj = [val for sublist in obj for val in sublist]
        #obj = [int(a) for a in obj]
        #obj.remove(ind)
        list.append(obj)
    return pd.DataFrame(list)
point_neighbour = neighbour_point(patch,res)

[0. 0.]
[0. 0.]
[0.         0.03225806]
[0.         0.03225806]
[0.         0.03225806]
[0.         0.06451613]
[0.         0.06451613]
[0.         0.06451613]
[0.         0.09677419]
[0.         0.09677419]
[0.         0.09677419]
[0.         0.12903226]
[0.         0.12903226]
[0.         0.12903226]
[0.         0.16129032]
[0.         0.16129032]
[0.         0.16129032]
[0.         0.19354839]
[0.         0.19354839]
[0.         0.19354839]
[0.         0.22580645]
[0.         0.22580645]
[0.         0.22580645]
[0.         0.25806452]
[0.         0.25806452]
[0.         0.25806452]
[0.         0.29032258]
[0.         0.29032258]
[0.         0.29032258]
[0.         0.32258065]
[0.         0.32258065]
[0.         0.32258065]
[0.         0.35483871]
[0.         0.35483871]
[0.         0.35483871]
[0.         0.38709677]
[0.         0.38709677]
[0.         0.38709677]
[0.         0.41935484]
[0.         0.41935484]
[0.         0.41935484]
[0.        0.4516129]
[0.        0.4516129]
[0. 

In [33]:
print(point_neighbour.head())

         0         1         2     3     4     5
0  [1, 33]  [32, 33]      None  None  None  None
1  [0, 33]   [2, 34]  [33, 34]  None  None  None
2  [1, 34]   [3, 35]  [34, 35]  None  None  None
3  [2, 35]   [4, 36]  [35, 36]  None  None  None
4  [3, 36]   [5, 37]  [36, 37]  None  None  None


In [55]:
encoder_input = tf.keras.Input(shape=(2))
x = layers.Dense(3, activation="relu", name="layer1")(encoder_input)
#x = layers.Dense(1, activation="relu")(x)
encoder_output = layers.Dense(1, name="end_layer")(x)

model = tf.keras.Model(encoder_input, encoder_output, name="encoder")

model.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
layer1 (Dense)               (None, 3)                 9         
_________________________________________________________________
end_layer (Dense)            (None, 1)                 4         
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [19]:
print(model.get_layer("layer1").get_weights())

a = np.array([[1,0,0],[-1,1,-1]])
b = np.array([0,0,0])
model.get_layer("layer1").set_weights([a,b])
print(model.get_layer("layer1").get_weights())

[array([[ 0.24995995,  0.8200555 ,  0.3698902 ],
       [-0.5026457 ,  0.01744807, -0.31461418]], dtype=float32), array([0., 0., 0.], dtype=float32)]
[array([[ 1.,  0.,  0.],
       [-1.,  1., -1.]], dtype=float32), array([0., 0., 0.], dtype=float32)]


In [29]:
print(model.get_layer("end_layer").get_weights())

a = np.array([1,1,-1]).reshape((3,1))
b = np.array([0])

model.get_layer("end_layer").set_weights([a,b])
print(model.get_layer("end_layer").get_weights())



[array([[-0.2690202],
       [-0.9547529],
       [ 0.0147301]], dtype=float32), array([0.], dtype=float32)]
[array([[ 1.],
       [ 1.],
       [-1.]], dtype=float32), array([0.], dtype=float32)]


In [49]:
tensor = tf.convert_to_tensor(np.array([3,2]).reshape(1,2))
print(float(model(tensor)))

3.0


In [111]:
def create_max_model(name1,name2):

    max_input = tf.keras.Input(shape=(2))
    x = layers.Dense(3, activation="relu", name="layer{}".format(name1))(max_input)
    max_output = layers.Dense(1, name="layer{}".format(name2))(x)
    max_model = tf.keras.Model(max_input, max_output, name="max")
    a = np.array([[1,0,0],[-1,1,-1]])
    b = np.array([0,0,0])
    max_model.get_layer("layer{}".format(name1)).set_weights([a,b])
    a = np.array([1,1,-1]).reshape((3,1))
    b = np.array([0])
    max_model.get_layer("layer{}".format(name2)).set_weights([a,b])
    return max_model#, max_input, max_output

a_model= create_max_model("a1","a2")
b_model = create_max_model("b1","b2")

#print(a_model(tensor))
#m_model.summary()

tf.Tensor([[3.]], shape=(1, 1), dtype=float32)


In [172]:
#TODO: define function that generates a model
#TODO: chain them


#encoder_input = tf.keras.Input(shape=(2))
x = layers.Concatenate()([a_model.output, b_model.output])
#x = layers.Concatenate()([a_model.output, layers.Input(1)])
#x = layers.Reshape((2,), name="layer_11")(x)
x = layers.Dense(3, activation="relu", name="layer1")(x)
larger_max = layers.Dense(1, name="layer_21")(x)

model_b = tf.keras.Model([a_model.input, b_model.input], larger_max, name="encoder")
#model_b = tf.keras.Model([a_model.input,  layers.Input(1)], larger_max, name="encoder")

a = np.array([[1,0,0],[-1,1,-1]])
b = np.array([0,0,0])
model_b.layers[-2].set_weights([a,b])

a = np.array([1,1,-1]).reshape((3,1))
b = np.array([0])
model_b.layers[-1].set_weights([a,b])


model_b.summary()
a1 = tf.convert_to_tensor( np.array([1,2]).reshape(1,2))
b1 = tf.convert_to_tensor( np.array([1,2]).reshape(1,2))

print(a_model(b1))
print(model_b([a1,b1]))
#tf.keras.utils.plot_model(model_b)

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           [(None, 2)]          0                                            
__________________________________________________________________________________________________
input_23 (InputLayer)           [(None, 2)]          0                                            
__________________________________________________________________________________________________
layera1 (Dense)                 (None, 3)            9           input_22[0][0]                   
__________________________________________________________________________________________________
layerb1 (Dense)                 (None, 3)            9           input_23[0][0]                   
____________________________________________________________________________________________

In [131]:
a_test = tf.convert_to_tensor( np.array([2.3]).reshape(1,1))
newIn1 = layers.Input(batch_shape = (1,1))
newOut1 = layers.Dense(1,activation=lambda xy: xy, name="a")(newIn1)
new_model = tf.keras.Model(newIn1, newOut1)
a = np.array([1]).reshape((1,1))
b = np.array([0])
new_model.get_layer("a").set_weights([a,b])
new_model.summary()

new_model(a_test)

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_38 (InputLayer)        [(1, 1)]                  0         
_________________________________________________________________
a (Dense)                    (1, 1)                    2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[2.3]], dtype=float32)>

In [186]:

def create_max_model_rev():

    max_input = tf.keras.Input(shape=(2))
    x = layers.Dense(3, activation="relu")(max_input)
    max_output = layers.Dense(1)(x)
    max_model = tf.keras.Model(max_input, max_output, name="max")
    a = np.array([[1,0,0],[-1,1,-1]])
    b = np.array([0,0,0])
    max_model.layers[1].set_weights([a,b])
    a = np.array([1,1,-1]).reshape((3,1))
    b = np.array([0])
    max_model.layers[2].set_weights([a,b])
    return max_model#, max_input, max_output



def recursive_max(array):
    length = len(array)
    index_counter = 0
    if length == 1:
        newIn1 = layers.Input(batch_shape = (1,1))
        newOut1 = layers.Dense(1,activation=lambda xy: xy)(newIn1)
        new_model = tf.keras.Model(newIn1, newOut1)
        a = np.array([1]).reshape((1,1))
        b = np.array([0])
        new_model.layers[1].set_weights([a,b])
        return new_model
    elif length == 2:
        new_model = create_max_model_rev()
        return new_model
    else:
        first_arr, second_arr = np.array_split(array,2)
        model_f = recursive_max(first_arr)
        model_s = recursive_max(second_arr)
        #model_f.summary()
        #model_s.summary()
        x = layers.Concatenate()([model_f.output, model_s.output])
        x = layers.Dense(3, activation="relu")(x)
        larger_max = layers.Dense(1)(x)
        model_max = tf.keras.Model([model_f.input, model_s.input], larger_max, name="encoder")
        # first layer
        a = np.array([[1,0,0],[-1,1,-1]])
        b = np.array([0,0,0])
        model_max.layers[-2].set_weights([a,b])

        # second layer
        a = np.array([1,1,-1]).reshape((3,1))
        b = np.array([0])
        model_max.layers[-1].set_weights([a,b])
        index_counter += 1
        return model_max


a_test = tf.convert_to_tensor( np.array([1,2]).reshape(1,2))
b_test = tf.convert_to_tensor( np.array([3,5]).reshape(1,2))

#print(model.layers)
#model = create_max_model_rev()
final_model = recursive_max([1,2,3,4,5])
tensor = tf.reshape( tf.convert_to_tensor( np.array([1,2,3,4,5])),final_model.input)
final_model.summary()
print(final_model.input)
#print(final_model([a_test,b_test]))

ValueError: Tried to convert 'shape' to a tensor and failed. Error: Dimension 1 in both shapes must be equal, but are 2 and 1. Shapes are [?,2] and [1,1].
	From merging shape 0 with other shapes. for '{{node tf.reshape/Reshape/packed/0}} = Pack[N=2, T=DT_FLOAT, axis=0](Placeholder, Placeholder_1)' with input shapes: [?,2], [1,1].

In [30]:
#df = pd.DataFrame([1,3])

arr = np.array([1,2,3])
arra = np.array([2,5,6])

np.append([arr],[arra],axis=0)
# TODO: copy the above split method, and structure the file!!!

array([[1, 2, 3],
       [2, 5, 6]])